# Flask web framework

Flask is a Python micro web framework created in 2010 (used by Pinterest and LinkedIn).  
The main difference with bigger web frameworks like Django is that its core is very minimalist : it does not come pre-packaged with a lot of functionalities, it does not require any other library, and allows granularity by importing only extensions for the features we need.

Flask has a lot of extensions offering various features such as validation, upload handling, authentication, database support...

### Installation

- Download the flask framework : `pip install flask`
- Ensure it is correctly installed : `flask --version`

### Hello World

To run a Flask webapp, we just need to create an instance of `Flask` and run it.  
To add routes to the webapp, we use the `app.route()` decorator.

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_handler():
    return 'Hello world'

if __name__ == '__main__':
    app.run(debug=True)

From a terminal, run it with `python app.py`, it should start the web server on port 5000 : http://127.0.0.1:5000/  
The `debug` mode allows to reload automatically the web server on code change.

### Path parameters

We can include som path parameters in the path of the decorator.  
The parameters name and type must be specified.  
The path parameters becomes available as function parameters in the handler function.

In [ ]:
@app.route('/users/<string:user_name>/posts/<int:post_id>')
def post_handler(user_name, post_id):
    return f'Fetching post ID {post_id} for user {user_name}'

### Restrict methods

We can restrict which HTTP methods are allowed for a given route.  
This is used a lot to allow GET and POST requests on a same route (GET to list the resources and POST to create one from a form for example).

In [ ]:
from flask import requests

@app.route('/users', methods=['GET', 'POST'])
def get_users():
    return 'Getting users with method %s' % request.method

The HTTP method of the request can be accessed via the `request` variable provided in the `flask` module.

### Templates

Instead of returning a string to the browser, Flask can use a template file with the `render_template` function.  
Template files must be under a `templates` sub-folder (this name is mandatory).

In [ ]:
@app.route('/home')
def home_handler():
    return render_template('home.html')

The Flask templates use the Jinja web templating engine.  
A template can inherit from another one, and contain some blocks that children templates can customize.  

Some common Jinja tags are :
- `{% block xxx %} ... {% endblock %}` to create a block that a child template can populate.
- `{% for x in my_list xxx %} ... {% endfor %}` to loop on all items of a Python list given as parameter to the template.
- `{% if condition %} ... {% else %} ... {% endif %}` to conditionally add some content in the template.


For example a `base.html` template for HTML pages could be :

In [ ]:
<!DOCTYPE HTML>
<html>
<head>
    <meta charset="utf-8" />
    <title> {% block title %} {% endblock %} </title>
</head>
<body>
    {% block body %} {% endblock %}
</body>
</html>

And a child template could extend this base template and populate the `title` and `body` blocks :

In [ ]:
{% extends 'base.html' %}

{% block title %}
Home
{% endblock %}

{% block body %}
<h1>Home page</h1>
{% endblock %}

Templates can receive parameters from the Python code.  
We can send a list of dict, and then loop on this list in the template with the `{% for x in my_list %}` Jinja tag.  
Some Python code can also be included in the template with the `{{ python code }}` notation.

_Flask handler :_

In [ ]:
@app.route('/friends')
def friends_handler():
    all_friends = [
        {'name': 'Bruce', 'hobby': 'tennis'},
        {'name': 'Peter', 'hobby': 'biology'},
        {'name': 'Damian', 'hobby': 'kung-fu', 'age': 12},
    ]
    return render_template('friends.html', friends=all_friends)

_friends.html template :_

In [ ]:
{% block body %}
<h2>Friends list :</h2>
<ul>
{% for friend in friends %}
    <li> {{ friend.name }} 
        {% if friend.age %}({{friend.age}} years old) {% endif %}
        who likes {{ friend.hobby }}
    </li>
{% endfor %}
</ul>
{% endblock %}

### Static files

Flask can serve static content, it needs to be added under a `static` sub-folder.  
The `static` sub-folder can then have a deeper hierarchy (for example contain folders for css, javascript, image files...)

Files under the `static` folder can be referenced from a template, for example :

In [ ]:
<link rel="stylesheet" href="static/css/app.css" />

The recommended way is to use the `url_for()` Flask method instead, that allows runtime parameters :

In [ ]:
<link rel="stylesheet" href="{{ url_for('static', filename='css/app.css') }}" />

### Database integration

Flask offers a nice integration with database layers via the `flask_sqlalchemy` module.  
SQLAlchemy lets us choose an SQL database technology (SQLite, mySQL, Postgres...), define the schema of our database and easily interact with the data using the _data mapper pattern_ (using classes as handlers to database tables).

- Install with `pip install flask_sqlalchemy`
- In the code set the `SQLALCHEMY_DATABASE_URI` variable to the path of the database and instanciate a `SQLAlchemy` database object

In [ ]:
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///posts.db'  # 3 slashes is for relative path
db = SQLAlchemy(app)

- Define tables as classes extending `db.Model`, with a Column field for each table column.  
  We can define a foreign key in a child table, and the corresponding relationship in the parent table.  
  For example :

In [ ]:
class BlogPost(db.Model):
    # fields look like class variables but they are converted to instance variables by SQLAlchemy
    id         = db.Column(db.Integer, primary_key=True)
    title      = db.Column(db.String(100), nullable=False)
    content    = db.Column(db.Text, nullable=False)
    created_on = db.Column(db.DateTime, nullable=False, default=datetime.now)
    
    # Field defined as foreign key
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

    def __repr__(self):
        return f'BlogPost({self.id}, {self.title}, {self.content}, {self.user_id}, {self.created_on})'


class User(db.Model):
    id            = db.Column(db.Integer, primary_key=True)
    username      = db.Column(db.String(20), unique=True, nullable=False)
    email         = db.Column(db.String(100), unique=True, nullable=False)
    image_file    = db.Column(db.String(20), nullable=False, default='default.jpg')
    password_hash = db.Column(db.String(60), nullable=False)
    
    # 1-to-many relationship : define a relationship in the parent and a foreign key in the child
    # calling user.posts will actually run a query to fetch the posts
    # calling post.author will get the User object associated to the foreign key
    # the 'lazy' will load the posts for a user only when requested
    posts = db.relationship('Post', backref='author', lazy=True)

    def __repr__(self):
        return f'User({self.id}, {self.username}, {self.email}, {self.image_file})'

- the database can be created once before we run the website with the `create_all()` method.  
  It reads all model classes defined in the file and creates the corresponding tables.  
  For SQLite, it creates a `posts.db` database file in the current folder.

In [ ]:
from app import db
db.create_all()   # create all tables defined as classes in the app.py file

- The database can then be accessed and populated by the web server URL handlers by using instances of the model classes.  
  For example, basic CRUD operations can be performed with :

In [ ]:
# read all blog posts in DB
res = BlogPost.query.all()

# filter the result
res = BlogPost.query.filter_by(title='Blog post 1').all()

# create a user in DB
db.session.add(User(username='user1', email='user1@blog.com', password_hash='hash1'))
db.session.commit()

# create a blog post in DB
db.session.add(BlogPost(title='Blog post 1', content='This is the content', user_id=1))
db.session.commit()

# get a blog post by ID
res = BlogPost.query.get(1)

# use pseudo-fields of the relationship just like normal fields
res = User.query.get(1).posts        # actually perform a query to retrieve the posts
res = BlogPost.query.get(1).author

# update a blog post by ID
post = BlogPost.query.get(1)
post.title += '!!!'
db.session.commit()

# delete a blog post by ID
db.session.delete(BlogPost.query.get(1))
db.session.commit()

### Bootstrap style

A webapp served with Flask can, just like any other webapp, use Bootstrap styles to improve its UI.  
As described on Bootstrap website https://getbootstrap.com, it only requires to add a `<link>` tag in the header and one or more `<script>` tag(s) at the bottom of the body to make all Bootstrap styles available to our web pages.

In the _Examples_ tab, the Bootstrap website offers many ready-to-use code snippets to include nice looking components in our pages, such as :
- Container block style with `class="container"`
- Button style with `class="btn btn-primary"`
- Navbar components with the `<nav>` tag
- Form components (input, text area, dropdown...)


### Forms with Flask


#### Native HTML Forms

It is possible to use forms only with standard HTML `input` fields and a `button` for submission :


In [ ]:
<form action="/posts/create" method="POST">
    <label for="title">Title</label>
    <input class="form-control" type="text" placeholder="Enter title" name="title" id="title">
    <br />
    <label for="content">Content</label>
    <textarea class="form-control" placeholder="Enter your post content" name="content" id="content"></textarea>
    <br />
    <input class="btn btn-primary" type="submit" value="Submit" />
</form>

When clicking on the Submit button, a POST request is sent to the specified URL, and the user values can be accessed in the Python handler with :

In [ ]:
post_title = request.form['title']
post_content = request.form['content']

#### Flask WT forms module

Flask can help with the form's fields and validation using the WT forms module.  
It can be installed with :

In [ ]:
pip install flask-wtf 
pip install email_validator

A Python class is created for each form with the fields and validators for each field.  
The form object is then given to the HTML template to easily generate the HTML code and validate the input.

Some custom validators can be added for a specific field `xxx` by simply adding a method called `validate_xxx` in the class.

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField
from wtforms.validators import DataRequired, Length, Email, EqualTo


# Form for the registration of a new user
class SignupForm(FlaskForm):
    username         = StringField('Username', validators=[DataRequired(), Length(min=5, max=20)])
    email            = StringField('Email', validators=[DataRequired(), Email()])
    password         = PasswordField('Password', validators=[DataRequired(), Length(min=6)])
    confirm_password = PasswordField('Confirm Password', validators=[DataRequired(), EqualTo("password")])
    remember         = BooleanField('Remember me')
    submit           = SubmitField('Sign up')

    # custom validator for the username field
    def validate_username(self, username):
    user = User.query.filter_by(username=username.data).first()
    if user:
        raise ValidationError('This username is already used, please choose a different one.')

The handler can then use this class to send to an HTML template :

In [ ]:
@app.route('/signup', methods=['GET', 'POST'])
def signup_handler():
    form = SignupForm()
    if form.validate_on_submit():
        flash(f'Account created for {form.username.data}!', 'success')
        return redirect(url_for('home_handler'))
    return render_template('signup.html', form=form)

This form can then be used by the template to generate the HTML tags and access the validation status.  
For example, the below form creates a label and an input for the username field, and displays any potential error.  
It uses some Bootstrap classes for the components style and the errors highlighting.

In [ ]:
<!-- empty action will post to the same route -->
<form method="POST" action="">
    <!-- required for security -->
    {{ form.hidden_tag() }}
    <fieldset class="form-group">

        <!-- Username field -->
        <div class="form-group">
            {{ form.username.label(class="form-control-label") }}
            {% if form.username.errors %}
                {{ form.username(class="form-control form-control-lg is-invalid") }}
                <div class="invalid-feedback">
                    {% for error in form.username.errors %}
                        <span>{{ error }}</span>
                    {% endfor %}
                </div>
            {% else %}
                {{ form.username(class="form-control form-control-lg") }}
            {% endif %}
        </div>

        [ ... same for other fields ...]

    </fieldset>
    <div class="form-group mt-3">
        {{ form.submit(class="btn btn-outline-info") }}
    </div>
</form>

### Structure of a Flask project

For a small project we can put all code in the same app.py file, but as the app grows we need to structure the code.  
A good approach is to put all the code of the app in its own package containing :
- a `__init__.py` file creating the `app` and the `db` variables and importing the routes
- a `routes.py` module defining all the available routes (importing `app` and `db`)
- a `models.py` module defining a class for each database table with SQLAlchemy
- a `forms.py` module defining a class for each form in the webapp with WTForm
- a `templates` folder containing all templates used by the routes
- a `static` folder containing all static files to serve (CSS, JS, images, ...)

Then the webapp can be started by a `run.py` containing only :

In [ ]:
from myblog import app

if __name__ == '__main__':
    app.run(debug=True)

If the app grows bigger, it can be good to create several sub-packages (split by logical domains of the app) within the webapp package.  
For a blog, we can for instance have :
- a `users` sub-package for routes and forms about user management (login/logout/password reset/account management...)
- a `posts` sub-package for routes and forms about post management (creation/update/deletion/list...)

Each sub-package is a folder under the main webapp package, containing :
- its own `__init__.py` file
- a `forms.py` file with forms for this module
- a `routes.py` file with forms for this module

The forms and routes files of each sub-package can import shared variables from the main module, like the `app` or the `db` instance.

Instead of defining directly the routes from the `app` object, each sub-package will define its own `Blueprint` containing all its routes:

In [ ]:
posts_blueprint = Blueprint('posts', __name__)


@posts_blueprint.route('/posts', methods=['GET'])
def get_posts_handler():
   ... logic to render the posts ...

Each call to `url_for()` will now need to specify the blueprint and the handler.  
For the above route, it is called with `url_for('posts.get_posts_handler')`.

The main webapp package will then import all blueprints in its `__init__.py` :

In [ ]:
from myblog.users.routes import users_blueprint
from myblog.posts.routes import posts_blueprint

app.register_blueprint(users_blueprint)
app.register_blueprint(posts_blueprint)

It is also a good practice to expose a `create_app(config)` function instead of creating directly an `app` object in the webapp package.  
That makes the app more testable and allows to instantiate it with different configs :
- create a Config class containing all the config variables of the app (secret ones may be read from env variables)
- create a `create_app(config)` function in the top-level `__init__.py` file
- in this function, create the app object and initialize it with the config class
- create the extensions outside of the `create_app` function (db, bcrypt, login_manager, mail...)
- initialize the extensions with the app instance inside the function with `db.init_app(app)` for instance.
- register all required blueprints for the app in the function

In [ ]:
def create_app(config_class=Config):
    # Create a Flask object representing our webapp
    app = Flask(__name__)
    app.config.from_object(config_class)

    # Extensions are created outside of the create_app method and are app-independent
    # Here we initialize them with our specific app object
    db.init_app(app)
    bcrypt.init_app(app)
    login_manager.init_app(app)
    mail.init_app(app)

    # register all blueprints for this app
    app.register_blueprint(users_blueprint)
    app.register_blueprint(posts_blueprint)
    app.register_blueprint(common_blueprint)

    return app


### User login

Login/logout management is made easy with the `flask-login` extension :  `pip install flask-login`

This extension allows to :
- call the `login_user(user)` function to login a user (it uses the session)
- call the `logout_user()` function to logout the current user
- access the `current_user` DB model variable from the Python code or the HTML templates (None if not logged in)
- restrict some routes to logged in users, and redirect to a different route if not logged in.  
  It adds a `next` query parameter to the redirected route, that can be used after login to redirect to the target page.

To get this extension to work, we need our User model class to extend the `UserMixin` class.  
This will create the fields required by the flask-login plugin (is_active, is_authenticated...).  
We also need to create a method to let the extension know how to create the user object from the user ID :

In [ ]:
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

### Image upload

Flask supports image upload by adding a field of type `FileField` to a form.  
This field can use the `FileAllowed()` validator to allow only some specific file extensions :

In [ ]:
picture = FileField('Update profile picture', validators=[FileAllowed(['jpg', 'png'])])

This field can then be used in the template receiving this form.  
Note that the form must specify an encoding type of `multipart/form-data` for the upload to be possible :

In [ ]:
<form method="POST" action="" enctype="multipart/form-data">
    <!-- required for security -->
    {{ form.hidden_tag() }}
    <fieldset class="form-group">

        [ ... other fields ... ]
    
        <!-- Profile picture field -->
        <div class="mt-3 form-group">
            {{ form.picture.label(class="form-control-label") }}
            <br />
            {{ form.picture(class="form-control-file") }}
            {% if form.picture.errors %}
                <div>
                    {% for error in form.picture.errors %}
                        <span class="text-danger">{{ error }}</span> <br />
                    {% endfor %}
                </div>
            {% endif %}
        </div>
    </fieldset>
</form>

On POST, that file can be retrieved and save on the server.  
For example, the below handler saves a new username, email and profile picture.  
If a new picture is uploaded, it creates a thumbnail of the received image and saves it on the server.  
It uses `Pillow` (Python Imaging Library) to resize the image :

In [ ]:
from PIL import Image # part of Pillow (Python Imaging Library)

def save_picture(picture):
    # create a name for the picture file
    hex = secrets.token_hex(8)
    _, file_ext = os.path.splitext(picture.filename)
    file_name = current_user.username + '_' + hex + file_ext
    picture_path = os.path.join(app.root_path, 'static/images', file_name)
    # Resize the image with Pillow
    output_size = (125, 125)
    i = Image.open(picture)
    i.thumbnail(output_size)
    # Save the file in the images folder
    i.save(picture_path)
    return file_name

@app.route('/account', methods=['GET', 'POST'])
def account_handler():
    form = UpdateAccountForm()
    if form.validate_on_submit():
        if form.picture.data:
            # save the file in the images folder
            file_name = save_picture(form.picture.data)
            # Update the DB with this file
            current_user.image_file = file_name
        current_user.username = form.username.data
        current_user.email = form.email.data
        db.session.commit()
        flash('Your account has been updated', 'success')
        return redirect(url_for('account_handler'))
    elif request.method == 'GET':
        # pre-populate the username and email
        form.username.data = current_user.username
        form.email.data = current_user.email

    image_path = url_for('static', filename='images/' + current_user.image_file)
    return render_template('account.html', title='Account', image_path = image_path, form=form)


### Sending emails

We may need to send email to our users from the Flask webapp (for example for password reset).  
This can be done with the `flask-mail` extension : `pip install flask-mail`

It requires to setup the mail server.  
The username and password should be passed via environment variables to remain secret.

In [ ]:
from flask_mail import Mail

app.config['MAIL_SERVER'] = 'smtp.googlemail.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = os.environ.get('EMAIL_USER')  
app.config['MAIL_PASSWORD'] = os.environ.get('EMAIL_PWD')
mail = Mail(app)

Then this mail server can be used to send emails from any route :

In [ ]:
def send_reset_email(user):
    msg = Message(subject='Password reset request',
                  recipients=[user.email],
                  body='This is a mail sent from Flask.',
                  sender='noreply@demo.com')
    mail.send(msg)

### HTTP errors handling

If an undefined route is queried, Flask will throw a 404 error.  
We can also throw a specific HTTP error from a route handler with `abort(error_code)`, for example a 403 if a user tries to update another user's resource.

By default, these errors will result in a simple HTML page with a black message on a white page.  
We can customize the error page to display a specific template, and include our webapp header/menu if needed.  

We can create a blueprint for the errors handling, and use the `@blueprint.app_errorhandler(error_code)` decorator for the handlers.  
The handler is similar to a normal route handler, it can render a template in the same way, but the return should include a second parameter for the error code.

In [ ]:
errors_blueprint = Blueprint('errors', __name__)

@errors_blueprint.app_errorhandler(404)
def error_404(error):
    return render_template('errors/404.html'), 404


### Deployment on AWS

Once the webapp behaves as expected in our local dev environment, we can deploy it to an actual production server.  
It could be running on an on-premise server, or on a cloud provider (AWS, GCP, Linode...).  
Below are the steps to deploy it on an AWS EC2 instance.

#### 1st step : Get the dev Flask webapp running on port 5000

- From an AWS console :
    - Launch a new EC2 instance with an SSH key pair
    - Update its security group to allow inbound traffic on ports 22 (SSH), 80 (HTTP) and 5000 (used by Flask for dev)
    - Note the public IP for this EC2 instance


- From our local webapp project folder :
    - Create a _requirements.txt_ file listing the project dependencies :  
      `pip3 freeze > requirements.txt`
    - Copy the entire project folder over to the home folder of the EC2 instance :  
      `scp -r -i private_ssh_key.pem <proj_folder> ec2-user@<EC2_IP>:~/`
    - SSH to the EC2 instance :  
      `ssh -i private_ssh_key.pem ec2-user@<EC2_IP>`  
      
      
- From the SSH session on the EC2 instance :
    - Update the packages and install python and pip :  
      `sudo yum update`  
      `sudo yum install python3 python3-pip`  
    - Create a Python virtual env, activate it and install all dependencies of the webapp in it :  
      `python3 -m venv <proj_folder>/venv`  
      `cd <proj_folder>`  
      `source venv/bin/activate`  
      `pip install -r requirements.txt`  
    - Start the webapp in dev mode :  
      `export FLASK_APP=run.py`  
      `flask run --host=0.0.0.0` &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (host `0.0.0.0` to not limit to localhost but listen to all public IPs)


- From any browser, ensure that the webapp is reachable at URL _http://\<EC2_IP\>:5000_

#### Step 2 : Get the app running with NGINX and Gunicorn

For a production environment, we want to support a high load of traffic.  
Running the app directly on the server does not allow that, we need to use a real web server like NGINX.  
NGINX will handle the traffic, deliver directly static files, and delegate the routes of the webapp to Gunicorn web server.  
Gunicorn will execute the Python code for the routes using several workers.

- Install NGINX and Gunicorn :  
  `sudo amazon-linux-extras install nginx1`  
  `pip install gunicorn` &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (run from within the Python virtual env)
  
  
- Create a _/etc/nginx/conf.d/flask_app.conf_ NGINX config file for the webapp.  
  All .conf files in this folder are imported in _nginx.conf_. &nbsp;  
  This config file tells NGINX to serve static files directly, and to delegate other requests to Gunicorn web server running on localhost.




In [ ]:
server {
        listen 80;
        server_name <EC2_IP>;

        location /static {
                alias /home/ec2-user/flask_app/myblog/static;
        }

        location / {
                proxy_pass http://localhost:8000;

                proxy_set_header Host $http_host;
                proxy_set_header X-Real-IP $remote_addr;
                proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
                proxy_set_header X-Forwarded-Proto $scheme;

                proxy_redirect off;
        }
}

- Restart NGINX web server :  
  `sudo systemctl restart nginx`
  
  
- Ensure you can access a static file from NGINX with URL : _http://\<EC2_IP\>/static/css/app.css_  
  **Note :** If you get a 403 error code (Forbidden), ensure the `nginx` user used by NGINX (as specified in _nginx.conf_) has read access to the static files and execute access to all its parent directories.
  
Now NGINX can serve static files, but we get a 502 error code (Bad Gateway) if we try to access the URL : _http://\<EC2_IP\>/home_ &nbsp;  
The Python routes are delegated by NGINX to the web server running on localhost, but we do not have one yet.  
We need to start Gunicorn web server on localhost to handle the Python routes.

To start Gunicorn with 3 parallel workers running our `app` flask instance from _run.py_ file, run :  
`cd <proj_folder>`  
`gunicorn -w 3 run:app`

Now we can access the URL : _http://\<EC2_IP\>/home_


#### Step 3 : Get the web server to run in the background with supervisor

The webapp is now functional, but Gunicorn is running in the foreground.  
If we press Ctrl-C or close our SSH terminal, Gunicorn stops and the webapp is no longer served.  
We can use `supervisor` to make it run in the background without the need for an open terminal.  

- Install supervisor :  
  `sudo amazon-linux-extras install epel` &nbsp;&nbsp;&nbsp;&nbsp; (add a new repository available that contains supervisor)  
  `sudo yum install supervisor`


- Create a supervisor config file called _/etc/supervisord.d/flask_app.ini_ for our webapp.  
  It is automatically included in _/etc/supervisord.conf_ supervisor main config file.  
  The webapp config file specifies how to start the Gunicorn webserver, where to save the logs and if it needs autostart.

In [ ]:
[program:flask_app]
directory=/home/ec2-user/flask_app
command=/home/ec2-user/flask_app/venv/bin/gunicorn -w 3 run:app
user=ec2-user
autostart=true
autorestart=true
stopasgroup=true
killasgroup=true
stderr_logfile=/var/log/flask_app/flask_app.err.log
stdout_logfile=/var/log/flask_app/flask_app.out.log

- Create the log folder and files specified in the above supervisor config file :  
  `sudo mkdir /var/log/flask_app`  
  `sudo touch /var/log/flask_app/flask_app.err.log`  
  `sudo touch /var/log/flask_app/flask_app.out.log`  


- Start supervisor and ensure the workers for our Flask app are running :  
  `sudo systemctl start supervisord`  
  `sudo systemctl enable supervisord`  
  `sudo systemctl status supervisord` &nbsp;&nbsp;&nbsp;&nbsp; (should show our workers)


- Close the SSH terminal and ensure the webapp is still available from a browser.

#### Step 4 : Additional deployment steps

For a real production server, other steps would be required, but are not Flask-specific :

**Domain Name :** Buy a domain name with a domain provider (AWS Route53, GoDaddy, NameCheap...), add the domain in Route53 and create a DNS A record routing to our EC2 instance

**TLS/SSL encryption :** Setup HTTPS
- In AWS the best option is to use an Elastic Load Balancer and configure it to use an SSL Certificate from AWS Certificate Manager.
- If not using an ELB, we can use a 3rd party certificate, for instance the `Let's encrypt` solution by installing an agent on our EC2 that will update our NGINX config

**Secrets Management :** Instead of hardcoding user/passwords or adding them in env variables, we could use AWS Secrets Manager and assign a role to the EC2 instance to access these secrets.